In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import datetime

from colour import Color

from autocorrect import Speller

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from datetime import date as dt

import folium
from folium import plugins
from folium.plugins import HeatMap
#http://geopandas.org/install.html
import geopandas as gpd

#import plotnine
#from plotnine import * 
#from IPython.display import display

import geopandas as gpd
from shapely.geometry import shape, Point

import requests
from pandas.io.json import json_normalize
import pandas_profiling

import warnings
warnings.filterwarnings('ignore')

In [6]:
inspections = pd.read_pickle('./datasets/cleaned_inspections.pickle')

In [7]:
inspections

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Community Area,Violation Numbers,Violation Comments,Reinspections
0,2345318,SUBWAY,SUBWAY,2529116,Restaurant,1,2620 N NARRAGANSETT AVE,2019-11-08,Canvass,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[3, 5, 58]",[ 2-102.14(O) OBSERVED NO WRITTEN EMPLOYEE HE...,1
1,2321166,SUBWAY,SUBWAY,2529116,Restaurant,1,2620 N NARRAGANSETT AVE,2019-11-06,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[3, 5, 10, 22, 33, 57, 58]",[ OBSERVED NO WRITTEN EMPLOYEE HEALTH POLICY O...,0
2,2290717,PAPA JOHN'S PIZZA,PAPA JOHN'S PIZZA,1545897,Restaurant,2,2620 N NARRAGANSETT AVE,2019-05-30,Canvass,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[3, 5]",[ OBSERVED NO EMPLOYEE HEALTH POLICY ON THE PR...,0
3,2144892,SUBWAY,SUBWAY,2529116,Restaurant,1,2620 N NARRAGANSETT AVE,2018-02-13,Canvass,Pass,NaN,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,NaN,NaN,1
4,2144588,SUBWAY,SUBWAY,2529116,Restaurant,1,2620 N NARRAGANSETT AVE,2018-02-06,Canvass,Fail,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",41.927995,-87.785752,"41.92799528871574, -87.78575236468352",BELMONT CRAGIN,"[16, 34, 42]",[ OBSERVED BLACK AND PINK MOLD LIKE SUBSTANCES...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195317,67784,"SURF & TURF WHERE SEAFOOD & STEAK MEET, INC.","SURF & TURF WHERE SEAFOOD & STEAK MEET, INC.",1869406,Restaurant,1,13205 S MUSKEGON AVE,2010-01-06,No Inspection,Fail,NaN,41.655351,-87.554280,"41.65535108240333, -87.554280206629",HEGEWISCH,NaN,NaN,0
195318,67848,SAFAH FOOD & LIQUOR INC,SAFAH FOOD & LIQUOR INC,14103,Market,3,7105 S RACINE AVE,2010-01-05,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.764896,-87.653965,"41.764896400247046, -87.65396483351302",ENGLEWOOD,NaN,NaN,1
195319,164256,SAFAH FOOD & LIQUOR INC,SAFAH FOOD & LIQUOR INC,14103,Market,3,7105 S RACINE AVE,2010-01-05,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.764896,-87.653965,"41.764896400247046, -87.65396483351302",ENGLEWOOD,NaN,NaN,1
195320,164255,RAINBOW GROCERY,RAINBOW GROCERY,61801,Market,3,2003 W 69TH ST,2010-01-05,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.768328,-87.673819,"41.768328334800714, -87.67381938402686",WEST ENGLEWOOD,NaN,NaN,1


In [8]:
# These are all Restaurant', 'Drinks Establishment', 'Bakery', 'Dessert Establishment', 'Coffeeshop', 'Vending
# that we will display in our map
establishments = pd.read_pickle('pickles/scores_features.pickle')
establishments

,DBA Name,Address,Community Area,Facility Type,Latitude,Longitude,Violation per Inspection,Critical Violation per Inspection,Moderate Violation per Inspection,Non-Critical Violation per Inspection,Critical Violations Ratio,Yes Ratio of VomitDiarrheal,Yes Ratio of Allergen,Yelp Rating,Critics Score
0,'LUNA EMPANADA SHOP',6011 W IRVING PARK RD,Portage Park,Restaurant,41.952934,-87.777228,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.000000
1,10 PIN BOWLING LOUNGE,330 N STATE ST,Loop,Restaurant,41.888361,-87.628180,0.105556,0.017857,0.050000,0.234375,0.052632,0.0,0.083333,0.7,0.941947
2,11 DEGREES NORTH,824 W BELMONT AVE,Lake View,Restaurant,41.940077,-87.650295,0.107407,0.011905,0.022222,0.270833,0.034483,0.0,0.000000,0.9,0.962296
3,14 PARISH,2333 S MICHIGAN AVE,Near South Side,Restaurant,41.850118,-87.623589,0.032099,0.007937,0.000000,0.083333,0.076923,0.0,0.000000,0.9,0.981739
4,1914 CLUB,1060 W ADDISON ST,Lake View,Restaurant,41.947317,-87.656418,0.053333,0.000000,0.013333,0.137500,0.000000,0.0,0.000000,0.9,0.983792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16783,morrison authentic southern cooking and pizzeria,8131 S ASHLAND AVE,Auburn Gresham,Restaurant,41.745778,-87.663176,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.000000
16784,mr.daniel's,5645 W BELMONT AVE,Belmont Cragin,Restaurant,41.938443,-87.768318,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.000000
16785,ropa restaurant & wine bar,1146 W PRATT BLVD,Rogers Park,Restaurant,42.005657,-87.660379,0.333333,0.285714,0.133333,0.562500,0.266667,1.0,1.000000,0.0,0.424732
16786,subway restaurant 1,3348 W lawrence AVE,Albany Park,Restaurant,41.968514,-87.712885,0.014815,0.000000,0.000000,0.041667,0.000000,0.0,0.000000,0.0,0.995694


## Let's try to only keep those that have a yelp rating

In [9]:
yelp_places = establishments[establishments['Yelp Rating'] != 0]
yelp_places

,DBA Name,Address,Community Area,Facility Type,Latitude,Longitude,Violation per Inspection,Critical Violation per Inspection,Moderate Violation per Inspection,Non-Critical Violation per Inspection,Critical Violations Ratio,Yes Ratio of VomitDiarrheal,Yes Ratio of Allergen,Yelp Rating,Critics Score
0,'LUNA EMPANADA SHOP',6011 W IRVING PARK RD,Portage Park,Restaurant,41.952934,-87.777228,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.000000
1,10 PIN BOWLING LOUNGE,330 N STATE ST,Loop,Restaurant,41.888361,-87.628180,0.105556,0.017857,0.050000,0.234375,0.052632,0.0,0.083333,0.7,0.941947
2,11 DEGREES NORTH,824 W BELMONT AVE,Lake View,Restaurant,41.940077,-87.650295,0.107407,0.011905,0.022222,0.270833,0.034483,0.0,0.000000,0.9,0.962296
3,14 PARISH,2333 S MICHIGAN AVE,Near South Side,Restaurant,41.850118,-87.623589,0.032099,0.007937,0.000000,0.083333,0.076923,0.0,0.000000,0.9,0.981739
4,1914 CLUB,1060 W ADDISON ST,Lake View,Restaurant,41.947317,-87.656418,0.053333,0.000000,0.013333,0.137500,0.000000,0.0,0.000000,0.9,0.983792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16672,YOBERRI,2814 1/2 N HALSTED ST,Lake View,Restaurant,41.933200,-87.649176,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.7,1.000000
16716,Yogen Fruz,333 S State ST BLDG,Loop,Restaurant,41.877397,-87.627515,0.095238,0.020408,0.028571,0.223214,0.066667,0.0,0.000000,0.7,0.960948
16717,Yuki Hana,2918-2920 N Clark ST,Lincoln Park,Restaurant,41.935287,-87.647072,0.056566,0.038961,0.024242,0.102273,0.214286,0.0,0.000000,0.6,0.954756
16727,ZACATECAS RESTAURANT,5037-5039 W DIVERSEY AVE,Belmont Cragin,Restaurant,41.931337,-87.752997,0.025926,0.000000,0.000000,0.072917,0.000000,0.0,0.000000,0.7,0.992465


In [10]:
y_locations_array = np.array(yelp_places[['DBA Name', 'Latitude','Longitude']])
y_locations_map = folium.Map(location=[41.86087, -87.608945], zoom_start=10)
for i, info in enumerate(y_locations_array):
    folium.Marker(
        location=[info[1], info[2]],
        popup=info[0],
        icon=folium.Icon(color='red', icon='info-sign')).add_to(y_locations_map)
y_locations_map.save('./docs/y_places.html')

The map is a bit too crowded

## Let's try to only plot by community area

Community Area: Portage Park

In [11]:
yelp_places_portage_park = yelp_places[yelp_places['Community Area']=='Portage Park']

In [12]:
y_locations_array_portage_park = np.array(yelp_places_portage_park[['DBA Name', 'Latitude','Longitude']])
y_locations_map_portage_park = folium.Map(location=[41.86087, -87.608945], zoom_start=10)
for i, info in enumerate(y_locations_array_portage_park):
    folium.Marker(
        location=[info[1], info[2]],
        popup=info[0],
        icon=folium.Icon(color='red', icon='info-sign')).add_to(y_locations_map_portage_park)
y_locations_map_portage_park.save('./docs/y_places_portage_park.html')

Community Area: Loop

In [13]:
yelp_places_loop = yelp_places[yelp_places['Community Area']=='Loop']

In [79]:
scores = np.sort(yelp_places_loop['Critics Score'].unique())
scores = [round(x,3) for x in scores]
unique_scores = [] 
[unique_scores.append(x) for x in scores if x not in unique_scores]
unique_scores=np.asarray(unique_scores)
unique_scores

array([0.628, 0.644, 0.658, 0.76 , 0.79 , 0.806, 0.813, 0.817, 0.819,
       0.822, 0.829, 0.832, 0.835, 0.836, 0.848, 0.853, 0.854, 0.856,
       0.857, 0.862, 0.864, 0.865, 0.874, 0.875, 0.878, 0.879, 0.88 ,
       0.882, 0.883, 0.889, 0.891, 0.892, 0.893, 0.896, 0.897, 0.901,
       0.903, 0.904, 0.907, 0.908, 0.909, 0.91 , 0.912, 0.913, 0.914,
       0.915, 0.918, 0.919, 0.92 , 0.921, 0.922, 0.925, 0.926, 0.927,
       0.928, 0.929, 0.931, 0.932, 0.933, 0.934, 0.935, 0.936, 0.937,
       0.938, 0.94 , 0.941, 0.942, 0.943, 0.944, 0.945, 0.946, 0.947,
       0.948, 0.949, 0.95 , 0.951, 0.952, 0.953, 0.954, 0.955, 0.956,
       0.957, 0.958, 0.959, 0.96 , 0.961, 0.962, 0.963, 0.964, 0.965,
       0.966, 0.967, 0.968, 0.969, 0.97 , 0.971, 0.972, 0.973, 0.974,
       0.975, 0.976, 0.977, 0.978, 0.979, 0.98 , 0.981, 0.982, 0.983,
       0.984, 0.985, 0.986, 0.987, 0.988, 0.989, 0.99 , 0.991, 0.992,
       0.993, 0.994, 0.995, 0.996, 0.997, 0.998, 0.999, 1.   ])

In [80]:
lime = Color("lime")
red = Color("red")
colour_range = list(red.range_to(lime, len(unique_scores)))
colour_range

[<Color red>,
 <Color #ff0400>,
 <Color #ff0800>,
 <Color #ff0c00>,
 <Color #ff1000>,
 <Color #ff1500>,
 <Color #ff1900>,
 <Color #ff1d00>,
 <Color #ff2100>,
 <Color #ff2500>,
 <Color #ff2900>,
 <Color #ff2d00>,
 <Color #ff3100>,
 <Color #ff3500>,
 <Color #ff3a00>,
 <Color #ff3e00>,
 <Color #ff4200>,
 <Color #ff4600>,
 <Color #ff4a00>,
 <Color #ff4e00>,
 <Color #ff5200>,
 <Color #ff5600>,
 <Color #ff5a00>,
 <Color #ff5f00>,
 <Color #ff6300>,
 <Color #ff6700>,
 <Color #ff6b00>,
 <Color #ff6f00>,
 <Color #ff7300>,
 <Color #f70>,
 <Color #ff7b00>,
 <Color #ff7f00>,
 <Color #ff8400>,
 <Color #f80>,
 <Color DarkOrange>,
 <Color #ff9000>,
 <Color #ff9400>,
 <Color #ff9800>,
 <Color #ff9c00>,
 <Color #ffa000>,
 <Color orange>,
 <Color #ffa900>,
 <Color #ffad00>,
 <Color #ffb100>,
 <Color #ffb500>,
 <Color #ffb900>,
 <Color #ffbd00>,
 <Color #ffc100>,
 <Color #ffc500>,
 <Color #ffca00>,
 <Color #ffce00>,
 <Color #ffd200>,
 <Color #ffd600>,
 <Color #ffda00>,
 <Color #ffde00>,
 <Color #ffe200>,


In [ ]:
y_locations_array_loop = np.array(yelp_places_loop[['DBA Name', 'Latitude','Longitude','Critics Score']])
y_locations_map_loop = folium.Map(location=[y_locations_array_loop[0][1], y_locations_array_loop[0][2]], zoom_start=15)
for i, info in enumerate(y_locations_array_loop):
    folium.CircleMarker(
        location=[info[1], info[2]],
        tooltip =(info[0],round(info[3],2)),
        radius = 6.5,
        weight=0,
        fill_color=colour_range[np.where(unique_scores == info[3])[0][0]].hex_l, fill_opacity=1).add_to(y_locations_map_loop)
    
y_locations_map_loop.save('./docs/y_places_loop.html')

In [17]:
y_locations_array_loop[0][1]

41.888361025678456

### Plotly

In [250]:
# scores = np.sort(yelp_places['Critics Score'].unique())
# scores = [round(x,3) for x in scores]
# unique_scores = [] 
# [unique_scores.append(x) for x in scores if x not in unique_scores]
# unique_scores=np.asarray(unique_scores)
# unique_scores

rounded_score = [round(score,3) for score in yelp_places['Critics Score']]
yelp_places['Score'] = rounded_score
yelp_places

,DBA Name,Address,Community Area,Facility Type,Latitude,Longitude,Violation per Inspection,Critical Violation per Inspection,Moderate Violation per Inspection,Non-Critical Violation per Inspection,Critical Violations Ratio,Yes Ratio of VomitDiarrheal,Yes Ratio of Allergen,Yelp Rating,Critics Score,marker_size,Score
0,'LUNA EMPANADA SHOP',6011 W IRVING PARK RD,Portage Park,Restaurant,41.952934,-87.777228,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.000000,1.00,1.000
1,10 PIN BOWLING LOUNGE,330 N STATE ST,Loop,Restaurant,41.888361,-87.628180,0.105556,0.017857,0.050000,0.234375,0.052632,0.0,0.083333,0.7,0.941947,0.15,0.942
2,11 DEGREES NORTH,824 W BELMONT AVE,Lake View,Restaurant,41.940077,-87.650295,0.107407,0.011905,0.022222,0.270833,0.034483,0.0,0.000000,0.9,0.962296,0.15,0.962
3,14 PARISH,2333 S MICHIGAN AVE,Near South Side,Restaurant,41.850118,-87.623589,0.032099,0.007937,0.000000,0.083333,0.076923,0.0,0.000000,0.9,0.981739,0.15,0.982
4,1914 CLUB,1060 W ADDISON ST,Lake View,Restaurant,41.947317,-87.656418,0.053333,0.000000,0.013333,0.137500,0.000000,0.0,0.000000,0.9,0.983792,0.15,0.984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16672,YOBERRI,2814 1/2 N HALSTED ST,Lake View,Restaurant,41.933200,-87.649176,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.7,1.000000,0.15,1.000
16716,Yogen Fruz,333 S State ST BLDG,Loop,Restaurant,41.877397,-87.627515,0.095238,0.020408,0.028571,0.223214,0.066667,0.0,0.000000,0.7,0.960948,0.15,0.961
16717,Yuki Hana,2918-2920 N Clark ST,Lincoln Park,Restaurant,41.935287,-87.647072,0.056566,0.038961,0.024242,0.102273,0.214286,0.0,0.000000,0.6,0.954756,0.15,0.955
16727,ZACATECAS RESTAURANT,5037-5039 W DIVERSEY AVE,Belmont Cragin,Restaurant,41.931337,-87.752997,0.025926,0.000000,0.000000,0.072917,0.000000,0.0,0.000000,0.7,0.992465,0.15,0.992


In [240]:
marker_size = [0.15 for score in yelp_places['Critics Score']]
yelp_places['marker_size'] = marker_size
yelp_places['marker_size'][0]= 1

In [259]:
fig = px.scatter_mapbox(yelp_places, lat="Latitude", lon="Longitude", hover_name="DBA Name",\
                        hover_data=["Score"], color="Score", color_continuous_midpoint = 0.82,\
                        color_continuous_scale=px.colors.diverging.RdYlGn,\
                        size = 'marker_size',
                        #size_max=30,
                        opacity=0.7,
                        zoom=10, height=600)
fig.update_layout(mapbox_style='carto-positron')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [255]:
fig.write_html('docs/scores_map.html')